### Gridsearching for the optimal hyerparameters

In this notebook, I used scikit-learn library GridsearchCV() method to find th optimal parameters. 
The hyperparameters include the number of convolutional layers, the number of filters, the pooling methods, the regularizer methods and alpha value, the number of dense layers and the number of nodes in each layer, and batch size. An exhausting computational power would be required to run all the hyperparameters in 1 gridsearch. That's why I ran more than 20 gridsearches and seeked for the optimal combinations. My decision criteria are 'time', 'accuracy', 'computational power', and 'loss'. Based on these criteria I found that 4 convolutional layers  and 2 dense layers with dropout regularizer is the optimal combination of hyperparameters.

In the below cells, you can see the full codes for some of the gridsearches.

In [ ]:
# Downloading the library packages
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.image as img

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from tensorflow.keras.regularizers import l1, l2

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import GridSearchCV

When we are finding the best parameters via gridsearch, used different photos which were obtained from shopping websites, such as macys.com and shop.nordstorm.com. These photos were saved in `train` and `test` folders.


In [ ]:
# In this cell I am creating the training data. (X_train and y_train)

# A target list that will contain target values
target_list = []

# A data list that will contain the image data. We will convert it to numpy array
data_list = []

# There are 44 photos in train folder. I want to randomly select them
ph_numbers = np.random.choice(np.linspace(1,44,44), 44, replace=False)

# In order to make sure all photos are in the same size, I defined the dimensions.
width = int(780)
height = int(1196)
dimensions = (width, height)

for i in ph_numbers:
    
    # reading each photo from the file. Also converting to float data type
    string = img.imread("train/"+str(int(i))+".jpeg").astype('float32')
    
    # squeshing the photo. (normalizing)
    string /= 255
    
    #resizing the photo using cv2
    string = cv2.resize(string, dimensions, interpolation = cv2.INTER_AREA) 
    
    # appending in a list where we keep all the photo data
    data_list.append(string)
    
    # if the photo names are in the below list, we know that they are "not liked", so we add 0 for them
    if i in [16, 3, 24, 37, 10, 26, 38, 11, 27, 39, 29, 40, 14, 41, 18, 34, 42, 22, 23, 36, 43 ,44]:
        target_list.append(0)
    # if the photo names are not in the above list, they are liked.
    else:
        target_list.append(1)
        

# converting the target list into numpy array
target_list = np.array(target_list)

# converting the image data list to a numpy array
X_train = np.array(data_list)

y_train = target_list

In [ ]:
# In this cell I amcreating the testing data. (X_test and y_test)

#loading the test photos, and converting them to float. Also normalizing by diviving by 255
t1 = img.imread('test/t7.jpeg').astype('float32') / 255
t2 = img.imread('test/t8.jpeg').astype('float32') / 255
t3 = img.imread('test/t9.jpeg').astype('float32') / 255
t4 = img.imread('test/t10.jpeg').astype('float32') / 255
t5 = img.imread('test/t11.jpeg').astype('float32') / 255
t6 = img.imread('test/t12.jpeg').astype('float32') / 255

#resizing the photos using OpenCV .cv2 function
t1 = cv2.resize(t1, dimensions, interpolation = cv2.INTER_AREA) 
t2 = cv2.resize(t2, dimensions, interpolation = cv2.INTER_AREA) 
t3 = cv2.resize(t3, dimensions, interpolation = cv2.INTER_AREA) 
t4 = cv2.resize(t4, dimensions, interpolation = cv2.INTER_AREA) 
t5 = cv2.resize(t5, dimensions, interpolation = cv2.INTER_AREA) 
t6 = cv2.resize(t6, dimensions, interpolation = cv2.INTER_AREA) 


# creating a numpy array of X variables
X_test = np.array([t1 , t2 , t3 , t4 , t5 , t6])

# creating target values, based on the numpy array sequence
y_test = np.array([0,1,1,1,0,0]) 


In [ ]:
# I define a function for the model. I will pass this into the gridsearch

def model_fn(layer_one_neurons=32, layer_two_neurons=32, layer_one_dropout=0.3, layer_two_dropout=0.3, 
             layer_three_neurons=32, layer_three_dropout=0.5):
    
    model = Sequential()
    
    # Add a convolutional layer.
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(1196, 780, 3)) )
    
    # add a pooling layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Add a convolutional layer.
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(1196, 780, 3)) )
    
    # add a pooling layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(layer_one_neurons, activation='relu'))
    model.add(Dropout(layer_one_dropout))
    
    model.add(Dense(layer_two_neurons, activation='relu'))
    model.add(Dropout(layer_two_dropout))
    
    model.add(Dense(layer_three_neurons, activation='relu'))
    model.add(Dropout(layer_three_dropout))
    
    model.add(Dense(2, activation="softmax"))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    return model

nn = KerasRegressor(build_fn=model_fn, batch_size=16, verbose=0)

In [ ]:
# The params grid:
nm_params = {
            'layer_one_neurons':[8, 16, 32, 64],
             'layer_two_neurons':[8, 16, 32, 64],
             'layer_one_dropout':[0.3, 0.4, 0.5],
            'layer_two_dropout':[0.3, 0.4, 0.5],
            'epochs':[10, 15, 20],
            'layer_three_neurons':[8, 16, 32, 64],
            'layer_two_dropout':[0.3, 0.4, 0.5]
            }

gs = GridSearchCV(estimator=nn, param_grid=nm_params, cv=2) # Use cv=3 just for the sake of time! 
gs.fit(X_train, y_train, verbose=0)

print(gs.best_score_)
gs.best_params_

In [ ]:
gs.cv_results_

In [ ]:
gs.predict(X_test)

In [ ]:
gs.score(X_test, y_test)

Since the required computational power is too high, I created few more gridsearch to try out different hyperparamaters. Especially for different reularizers.

In [ ]:
# Creating a function for a model with LASSO regularizer

def model_fn(layer_one_neurons=32, layer_two_neurons=32,layer_three_neurons=32, 
             layer_one_regularizer="l1(0.001)", layer_two_regularizer="l1(0.001)", layer_three_regularizer="l1(0.001)"):
    
    model = Sequential()
    
    # Add a convolutional layer.
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(1196, 780, 3)) )
    
    # add a pooling layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Add a convolutional layer.
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(1196, 780, 3)) )
    
    # add a pooling layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(layer_one_neurons, activation='relu', kernel_regularizer=layer_one_regularizer))
 
    
    model.add(Dense(layer_two_neurons, activation='relu', kernel_regularizer=layer_two_regularizer))
    
    model.add(Dense(layer_three_neurons, activation='relu', kernel_regularizer=layer_three_regularizer))
    
    model.add(Dense(2, activation="softmax"))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    return model

nn = KerasRegressor(build_fn=model_fn, batch_size=16, verbose=0)

In [ ]:
# The params grid:
nm_params = {
            'layer_one_neurons':[8, 16, 32, 64, 128],
             'layer_two_neurons':[8, 16, 32, 64, 128],
            'layer_three_neurons':[8, 16, 32, 64, 128],
             'layer_one_regularizer':["l1(0.001)", "l1(0.01)", "l1(0.0001)"],
            'layer_two_regularizer':["l1(0.001)", "l1(0.01)", "l1(0.0001)"],
            'layer_three_regularizer':["l1(0.001)", "l1(0.01)", "l1(0.0001)"],
            'epochs':[10, 15, 20]
            }

gs = GridSearchCV(estimator=nn, param_grid=nm_params, cv=2) # Use cv=3 just for the sake of time! 
gs.fit(X_train, y_train, verbose=0)

print(gs.best_score_)
gs.best_params_

In [ ]:
# Another model with Ridge regularizer

def model_fn(layer_one_neurons=32, layer_two_neurons=32,layer_three_neurons=32, 
             layer_one_regularizer="l2(0.001)", layer_two_regularizer="l2(0.001)", layer_three_regularizer="l2(0.001)"):
    
    model = Sequential()
    
    # Add a convolutional layer.
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(1196, 780, 3)) )
    
    # add a pooling layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Add a convolutional layer.
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(1196, 780, 3)) )
    
    # add a pooling layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(layer_one_neurons, activation='relu', kernel_regularizer=layer_one_regularizer))
 
    
    model.add(Dense(layer_two_neurons, activation='relu', kernel_regularizer=layer_two_regularizer))
    
    model.add(Dense(layer_three_neurons, activation='relu', kernel_regularizer=layer_three_regularizer))
    
    model.add(Dense(2, activation="softmax"))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    return model

nn = KerasRegressor(build_fn=model_fn, batch_size=16, verbose=0)

In [ ]:

# The params grid:
nm_params = {
            'layer_one_neurons':[8, 16, 32, 64, 128],
             'layer_two_neurons':[8, 16, 32, 64, 128],
            'layer_three_neurons':[8, 16, 32, 64, 128],
             'layer_one_regularizer':["l2(0.001)", "l2(0.01)", "l2(0.0001)"],
            'layer_two_regularizer':["l2(0.001)", "l2(0.01)", "l2(0.0001)"],
            'layer_three_regularizer':["l2(0.001)", "l2(0.01)", "l2(0.0001)"],
            'epochs':[10, 15, 20]
            }

gs = GridSearchCV(estimator=nn, param_grid=nm_params, cv=2) # Use cv=3 just for the sake of time! 
gs.fit(X_train, y_train, verbose=0)

print(gs.best_score_)
gs.best_params_